In [1]:
from langchain_community.document_loaders import JSONLoader

# oyunc dataları
player_loader = JSONLoader(
    file_path='../data/nba_fantasy_players.json',
    jq_schema='.[]',
    text_content=False
)
player_data = player_loader.load()

# takım dataları
team_loader = JSONLoader(
    file_path='../data/nba_league_summary.json',
    jq_schema='.[]',
    text_content=False
)
team_data = team_loader.load()

# tek liste haline getirme
data = player_data + team_data

In [2]:
print(f'toplam veri sayısı: {len(data)}, oyuncu veri sayısı: {len(player_data)}, takım veri sayısı: {len(team_data)}')

toplam veri sayısı: 452, oyuncu veri sayısı: 436, takım veri sayısı: 16


In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=0
)

docs = text_splitter.split_documents(data)

In [4]:
print(f"Number of documents after chunking: {len(docs)}")

Number of documents after chunking: 452


In [5]:
docs[12]

Document(metadata={'source': 'C:\\Users\\kaann\\Projects\\nba_fantasy_chatbot\\data\\nba_fantasy_players.json', 'seq_num': 13}, page_content='{"player_id": "6707", "name": "Tari Eason", "current_team": "Haramball", "position": "F/PF/SF", "AVG_PTS": 11.5, "AVG_REB": 5.1, "AVG_AST": 1.5, "AVG_ST": 1.1, "AVG_BLK": 0.6, "AVG_3PTM": 2.2, "AVG_TO": 1.5, "FG%": 0.507, "FT%": 0.6, "TOTAL_PTS": 184, "TOTAL_REB": 82, "TOTAL_AST": 24, "TOTAL_ST": 17, "TOTAL_BLK": 10, "TOTAL_3PTM": 35, "TOTAL_TO": 24, "FGM/A": "70/138", "FTM/A": "9/15"}')

In [6]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

c:\Users\kaann\anaconda3\envs\ortam_adi\lib\site-packages\google\api_core\_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.19) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
c:\Users\kaann\anaconda3\envs\ortam_adi\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
load_dotenv("../.env") 

True

In [8]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[-0.023955047130584717,
 0.011876456439495087,
 -0.003361367853358388,
 -0.058413900434970856,
 0.0015592977870255709]

In [9]:
from langchain_chroma import Chroma

In [10]:
import time
from langchain_chroma import Chroma

vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="../nba_fantasy_db"
)

# 2. Çok daha güvenli bir tempo ile yükleme (Batch: 10, Bekleme: 15sn)
batch_size = 10 
print(f"🚀 Toplam {len(docs)} döküman çok güvenli modda yükleniyor...")

for i in range(0, len(docs), batch_size):
    batch = docs[i : i + batch_size]
    try:
        vector_store.add_documents(documents=batch)
        print(f"✅ {i + len(batch)} / {len(docs)} tamamlandı...")
        
        # Ücretsiz plan dakikada 15 istek sınırı olabilir. 
        # 10'arlı gruplar ve 15 saniye bekleme ile dakikada 40 döküman işleriz.
        time.sleep(15) 
        
    except Exception as e:
        if "429" in str(e):
            print(f"⚠️ Kota doldu! 60 saniye dinleniyoruz...")
            time.sleep(60) # 429 hatasında 1 tam dakika bekle
            vector_store.add_documents(documents=batch) # Tekrar dene
            print(f"✅ Tekrar deneme başarılı: {i + len(batch)} / {len(docs)}")
        else:
            print(f"❌ Beklenmedik hata: {e}")

print("\n✨ TÜM VERİLER BAŞARIYLA YÜKLENDİ!")

🚀 Toplam 452 döküman çok güvenli modda yükleniyor...
✅ 10 / 452 tamamlandı...
✅ 20 / 452 tamamlandı...
✅ 30 / 452 tamamlandı...
✅ 40 / 452 tamamlandı...
✅ 50 / 452 tamamlandı...
✅ 60 / 452 tamamlandı...
✅ 70 / 452 tamamlandı...
✅ 80 / 452 tamamlandı...
✅ 90 / 452 tamamlandı...
✅ 100 / 452 tamamlandı...
✅ 110 / 452 tamamlandı...
✅ 120 / 452 tamamlandı...
✅ 130 / 452 tamamlandı...
✅ 140 / 452 tamamlandı...
✅ 150 / 452 tamamlandı...
✅ 160 / 452 tamamlandı...
✅ 170 / 452 tamamlandı...
✅ 180 / 452 tamamlandı...
✅ 190 / 452 tamamlandı...
✅ 200 / 452 tamamlandı...
✅ 210 / 452 tamamlandı...
✅ 220 / 452 tamamlandı...
✅ 230 / 452 tamamlandı...
✅ 240 / 452 tamamlandı...
✅ 250 / 452 tamamlandı...
✅ 260 / 452 tamamlandı...
✅ 270 / 452 tamamlandı...
✅ 280 / 452 tamamlandı...
✅ 290 / 452 tamamlandı...
✅ 300 / 452 tamamlandı...
✅ 310 / 452 tamamlandı...
✅ 320 / 452 tamamlandı...
✅ 330 / 452 tamamlandı...
✅ 340 / 452 tamamlandı...
✅ 350 / 452 tamamlandı...
✅ 360 / 452 tamamlandı...
✅ 370 / 452 tamamlan

In [ ]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":10})

In [26]:
retrieved_docs = retriever.invoke("Haramball takımının oyuncuları kimler?")

In [27]:
len(retrieved_docs)

20

In [28]:
print(retrieved_docs[9].page_content)

{"player_id": "6791", "name": "Dominick Barlow", "current_team": "Haramball", "position": "C/F/PF", "AVG_PTS": 8.6, "AVG_REB": 5.6, "AVG_AST": 1.9, "AVG_ST": 0.9, "AVG_BLK": 0.9, "AVG_3PTM": 0.4, "AVG_TO": 1.1, "FG%": 0.532, "FT%": 0.789, "TOTAL_PTS": 172, "TOTAL_REB": 111, "TOTAL_AST": 37, "TOTAL_ST": 18, "TOTAL_BLK": 18, "TOTAL_3PTM": 8, "TOTAL_TO": 22, "FGM/A": "67/126", "FTM/A": "30/38"}


In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash-lite",
    temperature=0.3,  
    max_tokens=500
)

In [30]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [31]:
system_prompt = (
    "You are a professional NBA Fantasy Basketball expert and strategy consultant. "
    "Use the provided context (player statistics, team ownership information, and league analysis) "
    "to answer user queries with high precision."
    "\n\n"
    "Strictly follow these rules:\n"
    "1. **Accuracy:** Use only the numerical data and facts found in the provided context. Do not hallucinate or use external NBA knowledge that isn't in the data.\n"
    "2. **Strategic Insight:** When asked about a team, analyze their strengths and weaknesses in core fantasy categories (PTS, REB, AST, STL, BLK, 3PM, etc.).\n"
    "3. **Missing Info:** If the answer is not present in the context, clearly state: 'I do not have current data regarding this player/team.'\n"
    "4. **Format:** Organize your responses with bullet points for readability, helping fantasy managers make quick decisions."
    "\n\n"
    "Context:\n"
    "{context}"
)

In [32]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("user", "{input}"),
    ]
)

In [33]:
question_answering_chain = create_stuff_documents_chain(llm,prompt)

In [34]:
rag_chain = create_retrieval_chain(retriever,question_answering_chain)

In [35]:
response = rag_chain.invoke({"input": "Doncic stats"})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 47.564153276s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_v

ResourceExhausted: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. To monitor your current usage, head to: https://ai.dev/usage?tab=rate-limit. 
* Quota exceeded for metric: generativelanguage.googleapis.com/generate_content_free_tier_requests, limit: 20, model: gemini-2.5-flash-lite
Please retry in 45.457763169s. [links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, violations {
  quota_metric: "generativelanguage.googleapis.com/generate_content_free_tier_requests"
  quota_id: "GenerateRequestsPerDayPerProjectPerModel-FreeTier"
  quota_dimensions {
    key: "model"
    value: "gemini-2.5-flash-lite"
  }
  quota_dimensions {
    key: "location"
    value: "global"
  }
  quota_value: 20
}
, retry_delay {
  seconds: 45
}
]

In [ ]:
print(response["answer"])

Here are Luka Dončić's stats:

*   **Average Points (AVG_PTS):** 33.7
*   **Average Rebounds (AVG_REB):** 8.3
*   **Average Assists (AVG_AST):** 8.6
*   **Average Steals (AVG_ST):** 1.6
*   **Average Blocks (AVG_BLK):** 0.6
*   **Average 3-Pointers Made (AVG_3PTM):** 3.4
*   **Average Turnovers (AVG_TO):** 4.3
*   **Field Goal Percentage (FG%):** 0.462
*   **Free Throw Percentage (FT%):** 0.803
*   **Total Points (TOTAL_PTS):** 775
*   **Total Rebounds (TOTAL_REB):** 191
*   **Total Assists (TOTAL_AST):** 198
*   **Total Steals (TOTAL_ST):** 37
*   **Total Blocks (TOTAL_BLK):** 14
*   **Total 3-Pointers Made (TOTAL_3PTM):** 79
*   **Total Turnovers (TOTAL_TO):** 99
*   **Field Goals Made/Attempted (FGM/A):** 240/520
*   **Free Throws Made/Attempted (FTM/A):** 216/269
